### Prepare files

Prepare first the list of files in the format `[id path duration(s)]`

In [ ]:
import sox
with open("vad.lst", "w") as fout, open("audio.lst", "r") as f:
  for n, name in enumerate(f):
    name = name.strip()
    fout.write("{}\t{}\t{}\n".format(
        n, name, sox.file_info.duration(name)))

In [ ]:
!head vad.lst

### Let's run VAD

In [ ]:
# lm can be skipped, then no perplexity value wil be in the result file
!./flashlight/build/bin/asr/fl_asr_voice_activity_detection_ctc \
   --am=am_transformer_ctc_stride3_letters_300Mparams.bin \
   --tokens=tokens.txt \
   --lexicon=lexicon.txt \
   --lm=lm_common_crawl_small_4gram_prun0-6-15_200kvocab.bin \
   --test=vad.lst \
   --outpath=vad-result

### Check VAD result

Let's have a look at results we have in the `outpath` directory

**Note:** number of frames for which we provide predictions is equal to `duration(seconds) * 1000 / stride(ms) / model_stride`, where stride is often 10ms for mfsc/mfcc features computation and model_stride in the example is 3.

In [ ]:
!ls vad-result/

In [ ]:
# id, percent of voice frames, perplexity of the transcription
!head vad-result/3.sts

In [ ]:
# final transcription
!head vad-result/3.tsc

In [ ]:
# per-frame transcription (# - is a blank token)
!head vad-result/3.fwt

In [ ]:
# per-frame blank confidence
!head vad-result/3.vad

**Result of VAD can be used to cut an audio into chunks for faster training/processing**

In [ ]:
with open("vad-result/3.fwt", "r") as f:
  plot_vad("audio/116-288045-0003.flac", f.readline().strip())